In [344]:
from dotenv import load_dotenv
import os
from os import environ as env
import psycopg2
import pandas as pd

In [345]:
# Load environment variables
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

In [346]:
# Connect to my MTG database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

In [347]:
from sqlalchemy import create_engine


# With the env variables loaded we can insert them into the engine connection string.
engine = create_engine(f"postgresql+psycopg2://{env['DB_USER']}:{env['DB_PASSWORD']}@{env['DB_HOST']}/{env['DB_NAME']}")
connection = engine.connect()

In [348]:
# Enter list of cards to match to a commander
items = ['Craterhoof Behemoth', 'Zur the Enchanter', 'Lightning Bolt', 'Dino DNA', 'Taniwha', 'Swords to Plowshares', 'Ancient Tomb']

# Create a comma-separated string of items
items_str = "', '".join(items)

query = f"SELECT DISTINCT coloridentity FROM cards WHERE name IN ('{items_str}');"
result_df = pd.read_sql(query, connection)

identities = [x.replace(',', '').replace(' ', '') for x in result_df['coloridentity'] if x != None]
identities_string = ''.join(identities)
command_color = list(''.join(dict.fromkeys(identities_string)))
command_color


['B', 'U', 'W', 'G', 'R']

In [349]:
color_conditions = ' AND '.join([f"coloridentity LIKE '%%{color}%%'" for color in command_color])

query = f"""
    SELECT DISTINCT name, edhrecrank, coloridentity, type, text, uuid 
    FROM (
        SELECT *
        FROM cards
        WHERE id in (select min(id) from cards group by name)
    ) unique_names
    WHERE type LIKE '%%Legendary%%Creature%%' 
        AND {color_conditions}
        AND edhrecrank >= 0
    ORDER BY edhrecrank
    LIMIT 100
    """

# Execute the query and store the results in a dataframe
commander_df = pd.read_sql(query, connection)

# Display the resulting dataframe
# commander_df


,name,edhrecrank,coloridentity,type,text,uuid
0,"Esika, God of the Tree // The Prismatic Bridge",1591,"B, G, R, U, W",Legendary Creature — God,Vigilance\n{T}: Add one mana of any color.\nOt...,51cea10e-7235-567e-9339-292571210ff5
1,"Morophon, the Boundless",1649,"B, G, R, U, W",Legendary Creature — Shapeshifter,Changeling (This card is every creature type.)...,2db25501-acbd-5eb3-bf02-20e9aaa3d132
2,"Sisay, Weatherlight Captain",1884,"B, G, R, U, W",Legendary Creature — Human Soldier,"Sisay, Weatherlight Captain gets +1/+1 for eac...",37a36baf-c9d2-5217-bd74-df614474344c
3,"Ramos, Dragon Engine",2153,"B, G, R, U, W",Legendary Artifact Creature — Dragon,"Flying\nWhenever you cast a spell, put a +1/+1...",606fda89-6450-5902-ba8c-3eb58e34b07e
4,"Kenrith, the Returned King",2191,"B, G, R, U, W",Legendary Creature — Human Noble,{R}: All creatures gain trample and haste unti...,b716ea42-01fa-5f8d-af42-efaecf77a748
5,The Ur-Dragon,2266,"B, G, R, U, W",Legendary Creature — Dragon Avatar,Eminence — As long as The Ur-Dragon is in the ...,7b10747d-fe9c-5ed2-9988-b9c55d41a2da
6,Tiamat,2448,"B, G, R, U, W",Legendary Creature — Dragon God,"Flying\nWhen Tiamat enters the battlefield, if...",e79b9335-6a28-52e2-8758-b150b3568967
7,"Jodah, the Unifier",2450,"B, G, R, U, W",Legendary Creature — Human Wizard,"Legendary creatures you control get +X/+X, whe...",04fae974-50e6-55e2-b05b-a0302b35fc8d
8,"Jodah, Archmage Eternal",2868,"B, G, R, U, W",Legendary Creature — Human Wizard,Flying\nYou may pay {W}{U}{B}{R}{G} rather tha...,6b063fe6-b14b-5699-8908-90f0679cbeb3
9,Sliver Hivelord,3008,"B, G, R, U, W",Legendary Creature — Sliver,Sliver creatures you control have indestructib...,b7bd9379-9347-5983-9957-b2d7536c057c


In [350]:
def score(creature_name):
    creature_name = creature_name.lower().replace('[^a-zA-Z0-9]', '').replace(' ', '-').replace(',', '').replace("'", '')
    score = 0
    url = f"https://json.edhrec.com/pages/commanders/{creature_name}.json"
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()
        entered_cards = []  # List to store entered cards that increased the score
        for entered_card in items:
            for edhrec_card in json_data['cardlist']:
                if entered_card == edhrec_card['name']:
                    score += 1
                    if edhrec_card['synergy'] >= 0.3:
                        score += 1
                    if edhrec_card['num_decks'] / edhrec_card['potential_decks'] >= 0.4:
                        score += 1
                    entered_cards.append(entered_card)  # Add entered card to the list
        return score, entered_cards
    else:
        return 0, []

commander_df['score'], commander_df['makesGoodUseOf'] = zip(*commander_df['name'].apply(score))


In [351]:
commander_df = commander_df.sort_values(['score', 'edhrecrank'], ascending=[False, True])
commander_df = commander_df.reset_index(drop=True)
top_10_df = commander_df[['score', 'edhrecrank', 'name', 'type', 'coloridentity', 'makesGoodUseOf']].head(10)
top_10_df


,score,edhrecrank,name,type,coloridentity,makesGoodUseOf
0,5,3108,Go-Shintai of Life's Origin,Legendary Enchantment Creature — Shrine,"B, G, R, U, W","[Zur the Enchanter, Swords to Plowshares]"
1,3,1649,"Morophon, the Boundless",Legendary Creature — Shapeshifter,"B, G, R, U, W","[Dino DNA, Swords to Plowshares, Ancient Tomb]"
2,3,1884,"Sisay, Weatherlight Captain",Legendary Creature — Human Soldier,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
3,3,2191,"Kenrith, the Returned King",Legendary Creature — Human Noble,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
4,3,2266,The Ur-Dragon,Legendary Creature — Dragon Avatar,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
5,3,2450,"Jodah, the Unifier",Legendary Creature — Human Wizard,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
6,3,3453,"Najeela, the Blade-Blossom",Legendary Creature — Human Warrior,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
7,3,3690,Scion of the Ur-Dragon,Legendary Creature — Dragon Avatar,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
8,3,5729,"Urtet, Remnant of Memnarch",Legendary Artifact Creature — Myr,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
9,3,5736,Sliver Queen,Legendary Creature — Sliver,"B, G, R, U, W","[Swords to Plowshares, Ancient Tomb]"
